In [1]:
from nba_api.stats.endpoints import commonallplayers
import pandas as pd

# Fetch all players (not only the current season)
common_all_players = commonallplayers.CommonAllPlayers(
    is_only_current_season=0,  # 1 active, 0 not active
    league_id='00'  # NBA 00, G League 20, WNBA 10
)

# Get the DataFrame
df_common_players = common_all_players.get_data_frames()[0]

# Convert FROM_YEAR and TO_YEAR to integers
df_common_players['FROM_YEAR'] = df_common_players['FROM_YEAR'].astype(int)
df_common_players['TO_YEAR'] = df_common_players['TO_YEAR'].astype(int)

# Filter for players active in the 2022-2023 season
df_common_players_2022_2023 = df_common_players[(df_common_players['FROM_YEAR'] <= 2022) & (df_common_players['TO_YEAR'] >= 2023)]

# Display the first few rows of the filtered DataFrame
print(df_common_players_2022_2023.head())

# Save the filtered DataFrame to a CSV file
df_common_players_2022_2023.to_csv('players_2022_2023.csv', index=False)
print("Player names and teams for the 2022-2023 season saved successfully!")


    PERSON_ID DISPLAY_LAST_COMMA_FIRST DISPLAY_FIRST_LAST  ROSTERSTATUS  \
10    1630173        Achiuwa, Precious   Precious Achiuwa             1   
22     203500            Adams, Steven       Steven Adams             1   
24    1628389             Adebayo, Bam        Bam Adebayo             1   
29    1630534            Agbaji, Ochai       Ochai Agbaji             1   
42    1630583            Aldama, Santi       Santi Aldama             1   

    FROM_YEAR  TO_YEAR        PLAYERCODE       PLAYER_SLUG     TEAM_ID  \
10       2020     2024  precious_achiuwa  precious_achiuwa  1610612752   
22       2013     2024      steven_adams      steven_adams  1610612745   
24       2017     2024       bam_adebayo       bam_adebayo  1610612748   
29       2022     2024      ochai_agbaji      ochai_agbaji  1610612761   
42       2021     2024      santi_aldama      santi_aldama  1610612763   

   TEAM_CITY  TEAM_NAME TEAM_ABBREVIATION  TEAM_CODE  TEAM_SLUG  \
10  New York     Knicks              

In [1]:
from nba_api.stats.endpoints import commonallplayers, commonplayerinfo
import pandas as pd
import concurrent.futures

# Fetch all players (not only the current season)
common_all_players = commonallplayers.CommonAllPlayers(
    is_only_current_season=0,  # 1 active, 0 not active
    league_id='00'  # NBA 00, G League 20, WNBA 10
)

# Get the DataFrame
df_common_players = common_all_players.get_data_frames()[0]

# Convert FROM_YEAR and TO_YEAR to integers
df_common_players['FROM_YEAR'] = df_common_players['FROM_YEAR'].astype(int)
df_common_players['TO_YEAR'] = df_common_players['TO_YEAR'].astype(int)

# Filter for players active in the 2022-2023 season
df_common_players_2022_2023 = df_common_players[(df_common_players['FROM_YEAR'] <= 2022) & (df_common_players['TO_YEAR'] >= 2023)]

# Fetch player info
def fetch_player_info(player_id):
    try:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()
        common_player_info = player_info[0]  # Get the CommonPlayerInfo DataFrame
        player_headline_stats = player_info[1]  # Get the PlayerHeadlineStats DataFrame

        # Combine the required information from both DataFrames
        player_data = {
            'player_id': player_id,
            'first_name': common_player_info.loc[0, 'FIRST_NAME'],
            'last_name': common_player_info.loc[0, 'LAST_NAME'],
            'team': common_player_info.loc[0, 'TEAM_NAME'],
            'position': common_player_info.loc[0, 'POSITION'],
            'height': common_player_info.loc[0, 'HEIGHT'],
            'weight': common_player_info.loc[0, 'WEIGHT'],
            'pts': player_headline_stats.loc[0, 'PTS'],
            'ast': player_headline_stats.loc[0, 'AST'],
            'reb': player_headline_stats.loc[0, 'REB'],
            'pie': player_headline_stats.loc[0, 'PIE'],
        }
        return player_data
    except Exception as e:
        print(f"An error occurred while fetching player info for player ID {player_id}: {e}")
        return None

# Get all unique player IDs from the filtered players
player_ids = df_common_players_2022_2023['PERSON_ID'].unique()

# Initialize a list to store player information
player_info_list = []

# Use ThreadPoolExecutor to fetch player info in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    player_infos = list(executor.map(fetch_player_info, player_ids))
    player_infos = [info for info in player_infos if info is not None]  # Filter out None results

# Convert the list of player info to a DataFrame
player_info_df = pd.DataFrame(player_infos)

# Display the first few rows of the DataFrame
print(player_info_df.head())

# Save the player info to a CSV file
player_info_df.to_csv('player_names_teams_positions_2022_2023.csv', index=False)
print("Player names, teams, positions, and stats for the 2022-2023 season saved successfully!")


An error occurred while fetching player info for player ID 1627853: 'PIE'
An error occurred while fetching player info for player ID 1631230: 'PIE'
An error occurred while fetching player info for player ID 1629628: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while fetching player info for player ID 201587: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while fetching player info for player ID 1630567: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred while fetching player info for player ID 1628966: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred while fetching player info for player ID 1629646: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred while fetching player info for player ID 1629647